In [1]:
# PATHS TO FILES

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [2]:
xp.init('C:/xpressmp/bin/xpauth.xpr')
pd.options.mode.copy_on_write = True

In [3]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)


In [4]:
# Slot duration 30 minutes
slot_duration = 30  # minutes
movies_df['slots_needed'] = (movies_df['runtime_with_ads'] / slot_duration).apply(lambda x: int(x)).astype(int)


In [5]:
# Define the broadcasting date

k = 2
start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
test_range = pd.date_range(start_date, periods = k)

broadcast_date = []
for i in test_range:
    broadcast_date.append(i)
for i in broadcast_date:
    broadcast_start = i.replace(hour=7, minute=0)
    broadcast_end = i.replace(hour=23, minute=30)
# print(broadcast_date)
    

# Generate all time slots
 
time_slots = []
current_time = broadcast_start
while current_time <= broadcast_end:
    time_slots.append(current_time)
    current_time += timedelta(minutes=slot_duration)

# Create mappings between time slots and indices
time_to_index = {t: idx for idx, t in enumerate(time_slots)}
index_to_time = {idx: t for idx, t in enumerate(time_slots)}


In [6]:
# Set 'Date' as index
#ch_A_schedule.set_index('Date', inplace=True)

# Resample to 30-minute intervals (use mean of groups)
ch_A_schedule_30min = ch_A_schedule.resample('30T').mean(numeric_only = True).reset_index()



C:\Users\Elie\AppData\Local\Temp\ipykernel_5032\290806634.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_A_schedule_30min = ch_A_schedule.resample('30T').mean(numeric_only = True).reset_index()


In [7]:
# # Create a dictionary to hold baseline viewership per time slot
# baseline_viewership = {}

# for idx, row in ch_A_schedule_30min.iterrows():
#     time_slot_time = row['Date'].time()
#     baseline_viewership[time_slot_time] = {
#         'children': row['children_baseline_view_count'],
#         'adults': row['adults_baseline_view_count'],
#         'retirees': row['retirees_baseline_view_count'],
#         'prime_time_factor': row.get('prime_time_factor', 1)
#     }


In [8]:
# Check for duplicate movie titles
duplicate_titles = movies_df[movies_df.duplicated(subset=['title'], keep=False)]
if not duplicate_titles.empty:
    print("Duplicate movie titles found:")
    print(duplicate_titles['title'])
else:
    print("No duplicate movie titles found.")


Duplicate movie titles found:
4                 The Avengers
17                     Titanic
76               The Lion King
105       Beauty and the Beast
149        Alice in Wonderland
                 ...          
5748              Midnight Sun
5761                The Island
5773            The Shaggy Dog
5855    Fun with Dick and Jane
5879        The Perfect Weapon
Name: title, Length: 258, dtype: object


In [9]:
# print("Baseline Viewership Data:")
# for key, value in list(baseline_viewership.items())[:5]:  # Print first 5 entries
#     print(f"Time Slot {key}: {value}")


In [10]:
# Total population (adjust as needed)
total_population = 1_000_000 

In [11]:
# # PRINT EXAMPLE OF BASE VIEWERSHIP. CONTAINS VIEWERSHIP PER MOVIE PER TIME SLOT (sums all groups)
# print("Sample of base_viewership:")
# sample_items = list(base_viewership.items())[:5]
# for key, value in sample_items:
#     print(f"{key}: {value}")


In [52]:
movies_small = movies_df.sample(150)
movies_small.set_index('title', inplace = True)
num_movies = len(movies_small)
num_slots = 34

mov = range(num_movies)
sched = range(num_slots)

In [53]:
movies_small.head()

,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres,n_ad_breaks,runtime_with_ads,scaled_popularity,children_scaled_popularity,adults_scaled_popularity,retirees_scaled_popularity,slots_needed
title,,,,,,,,,,,,,,,
Simply Irresistible,5.371,214,1999-02-05,4398989,95,6000000,10.873,"['Comedy', 'Romance']",4,120.0,0.240076,0.192061,0.240076,0.240076,4
Smart People,5.851,235,2008-04-11,10569964,95,7000000,6.908,"['Comedy', 'Drama', 'Romance']",4,120.0,0.017873,0.010724,0.017873,0.017873,4
Jay and Silent Bob Reboot,5.673,463,2019-10-15,1011305,105,10000000,13.760,['Comedy'],4,120.0,0.344848,0.344848,0.344848,0.344848,4
10 Things I Hate About You,7.580,7386,1999-03-30,53478166,97,16000000,36.364,"['Comedy', 'Romance', 'Drama']",4,120.0,0.715026,0.429015,0.715026,0.715026,4
Tremors,6.878,2935,1990-01-19,48572000,96,11000000,21.915,"['Horror', 'Action', 'Science Fiction']",4,120.0,0.533505,0.320103,0.533505,0.213402,4


In [62]:
def model(T, movies):
    prob = xp.problem(name="Movie_Scheduling_Problem")
    # M = range(len(movies_small))
    #print(movies.loc['Lethal Weapon 2', 'runtime_with_ads'])
    # Decision Variables
    # if movie m in shown in time slot t
    x = {(m,t): xp.var(vartype=xp.binary, name='x{0}_{1}'.format(m,t)) 
         for t in T for m in movies}
    prob.addVariable(x)
    # for t in T:
    #     print(x['Lethal Weapon 2',t])
    # if movie m is shown 
    y = {(m): xp.var(vartype=xp.binary, name='y{0}'.format(m)) for m in movies}
    prob.addVariable(y)

    # start time of movie m 
    s = {(m): xp.var(vartype=xp.integer, name='s{0}'.format(m)) for m in movies}
    prob.addVariable(s)

    # end time of movie m 
    e = {(m): xp.var(vartype=xp.integer, name='e{0}'.format(m)) for m in movies}
    prob.addVariable(e)

    # movie duration
    

    # Constraints
    # for t in T:
        # big M 
    M = len(time_slots)
    T_end = len(time_slots) -1 # last time slot index
    
    # movie duration 
    prob.addConstraint(xp.Sum(x[m,t] for t in T) == movies.loc[m, 'runtime_with_ads']*y[m] for m in movies)

    # end time
    # prob.addConstraint(e[m] == t*xp.Sum(x) for m in movie.index())

    # end time limit
    prob.addConstraint((t+1)*x[m,t] <= e[m] for m in movies for t in T)

    # start time limit
    #prob.addConstraint(s[m] <= (t*x[m,t]) for m in movie.index() fot t in T)

    # latest time a movie can start and still show full movie 
    prob.addConstraint(s[m] <= t*x[m,t] + (1-x[m,t])*M for m in movies)

    # end-start = movie duration
    prob.addConstraint(e[m]-s[m] == movies.loc[m, 'runtime_with_ads']*y[m] for m in movies)

    # last movie time?
    prob.addConstraint(s+d-1 <= T_end)

    return prob

In [63]:
days = 1
total_schedule = []
for k in range(1): 
    T = range(k*33, (k+1)*33+1)

    prob =  model(T,movies_small)
    
    prob.solve()
    
    scheduled_movies, used_movie_ids = get_sched(prob,movies_small)

    for i in used_movie_ids:
        movies_small.drop(i, inplace = True)
    total_schedule.append(scheduled_movies)

KeyError: 'vote_average'

In [47]:
movies_small.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres,n_ad_breaks,runtime_with_ads,scaled_popularity,children_scaled_popularity,adults_scaled_popularity,retirees_scaled_popularity,slots_needed
1346,Lethal Weapon 2,7.006,3092,1989-07-07,227853986,114,25000000,25.785,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4,120.0,0.594276,0.475420,0.594275,0.356565,4
3130,Ghosts of Mars,5.127,977,2001-08-24,14010832,98,28000000,14.189,"['Action', 'Horror', 'Science Fiction']",4,120.0,0.358020,0.214812,0.358020,0.143208,4
2859,Shaft,5.970,1162,2000-06-15,107626125,99,46000000,18.997,"['Action', 'Adventure', 'Crime', 'Thriller']",4,120.0,0.478001,0.382401,0.478001,0.286800,4
3542,Alfie,5.792,747,2004-10-22,35060882,103,60000000,15.451,"['Comedy', 'Drama', 'Romance']",4,120.0,0.394016,0.236409,0.394016,0.394016,4
2271,Snakes on a Plane,5.415,1614,2006-08-17,62022014,105,33000000,26.450,"['Action', 'Crime', 'Thriller', 'Adventure']",4,120.0,0.603570,0.482856,0.603570,0.362142,4


In [ ]:
movies.loc[